In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Reshape, Dropout
from keras.layers.normalization import BatchNormalization as BN
from keras.layers import GaussianNoise as GN
from keras.optimizers import SGD
from keras.callbacks import CSVLogger, ModelCheckpoint
from keras.callbacks import LearningRateScheduler as LRS
from keras.callbacks import ReduceLROnPlateau as RLRP
from keras.preprocessing.image import ImageDataGenerator

In [2]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [3]:
batch_size = 64
epochs = 200
num_classes=10

In [4]:
def configure_callbacks(model_id, kpi_to_monitor='val_accuracy'):
    # Without log/ or models/ subfolder as not possible to access unexisting folders
    # If possible to train with jupyter revise
    name = "mlp"
    log_filename = 'models/%s-%s.log' % (name, model_id)
    csv_logger = CSVLogger(log_filename)
    
    chk_1_model_filename = 'models/%s-%s-{epoch:04d}-{%s:.6f}.h5' % (name, model_id, kpi_to_monitor)
    chk_2_model_filename = 'models/%s-%s.h5' % (name, model_id)
    
    # Save best model fully not only weights after each epoch (period=1) 
    # with best accuracy value (mode=max, save_best_only=True)
    checkpoint1 = ModelCheckpoint(
        chk_1_model_filename,
        monitor=kpi_to_monitor,
        save_best_only=True,
        save_weights_only=False, 
        verbose=1, mode='max', period=1
    )

    checkpoint2 = ModelCheckpoint(
        chk_2_model_filename, 
        monitor=kpi_to_monitor,
        save_best_only=False,
        save_weights_only=False, 
        verbose=1, mode='auto', period=1
    )

    callbacks = [csv_logger, checkpoint1, checkpoint2]
    return callbacks

In [5]:
# Define a learning rate scheduler
def scheduler(epoch):
    if epoch < 25:
        return 0.1
    elif epoch < 50:
        return 0.01
    else:
        return 0.001

scheduler_lr = LRS(scheduler)

In [6]:
# Define a learning rate scheduler
def scheduler_2(epoch):
    if epoch < 75:
        return 0.1
    elif epoch < 125:
        return 0.01
    else:
        return 0.001

scheduler_lr_2 = LRS(scheduler)

In [7]:
# Define an automatic learning rate scheduler
reduce_lr = RLRP(monitor='val_loss', factor=0.5, cooldown=1,
                 patience=10, min_lr=0.001)

In [8]:
# Data Augmentation with an ImageGenerator
datagen = ImageDataGenerator(width_shift_range=0.1,
                             height_shift_range=0.1,
                             horizontal_flip=False,
                             vertical_flip=False)

In [9]:
# Data Augmentation with an ImageGenerator + more variations
datagen_plus = ImageDataGenerator(width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  horizontal_flip=False,
                                  vertical_flip=False,
                                  zoom_range=[1.0,1.1],
                                  rotation_range=10.0)

In [10]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('training set', x_train.shape)
print('test set', x_test.shape)

# Obtain as 28x28 for DataAugmentation, then reshape to 784
x_train = x_train.reshape(60000,28,28,1)
x_test = x_test.reshape(10000,28,28,1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize [0..255]-->[0..1]
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

training set (60000, 28, 28)
test set (10000, 28, 28)


In [11]:
# Optimizer 
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9)

In [12]:
def basic_training_and_evaluation(model, model_id, scheduler):
    # Compile Model
    model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    
    history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                        steps_per_epoch=len(x_train)/batch_size,
                        epochs=epochs,
                        shuffle=True,
                        verbose=1,
                        validation_data=(x_test, y_test),                   
                        callbacks=[configure_callbacks(model_id), scheduler])
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', loss)
    print('Test accuracy:', accuracy)
    print('Test error:', (1-accuracy))
    
    return model

In [13]:
def advanced_training_and_evaluation(model, model_id, scheduler):
    # Compile Model
    model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    
    history = model.fit(datagen_plus.flow(x_train, y_train, batch_size=batch_size),
                        steps_per_epoch=len(x_train)/batch_size,
                        epochs=epochs,
                        shuffle=True,
                        verbose=1,
                        validation_data=(x_test, y_test),                   
                        callbacks=[configure_callbacks(model_id), scheduler])
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', loss)
    print('Test accuracy:', accuracy)
    print('Test error:', (1-accuracy))
    
    return model

## Extra funcitons

In [ ]:
# Compile Model
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [ ]:
history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train)/batch_size,
                    epochs=epochs,
                    shuffle=True,
                    verbose=1,
                    validation_data=(x_test, y_test),                   
                    callbacks=[configure_callbacks(model_id), scheduler_lr])

In [ ]:
history = model.fit(datagen_plus.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train)/batch_size,
                    epochs=epochs,
                    shuffle=True,
                    verbose=1,
                    validation_data=(x_test, y_test),                   
                    callbacks=[configure_callbacks(model_id), reduce_lr])

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)
print('Test error:', (1-accuracy))

In [ ]:
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=False)
history = pd.read_csv("model-weights/model.log")
model = load_model("model-weights/model.h5")

## Models

In [16]:
# Model Basic
model = Sequential()
model.add(Reshape(target_shape=(784,), input_shape=(28,28,1)))
model.add(GN(0.3))
#model.add(Activation('relu'))

model.add(Dense(1024, activation=None))
model.add(BN())
model.add(GN(0.3))
model.add(Activation('relu'))

model.add(Dense(1024, activation=None))
model.add(BN())
model.add(GN(0.3))
model.add(Activation('relu'))

model.add(Dense(1024, activation=None))
model.add(BN())
model.add(GN(0.3))
model.add(Activation('relu'))

model.add(Dense(num_classes, activation='softmax'))

model.summary()
model_basic = model

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 784)               0         
_________________________________________________________________
gaussian_noise (GaussianNois (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 1024)              803840    
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 1024)              0         
_________________________________________________________________
activation (Activation)      (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1

In [17]:
# Model Dropout
model = Sequential()
model.add(Reshape(target_shape=(784,), input_shape=(28,28,1)))
model.add(GN(0.3))
#model.add(Activation('relu'))

model.add(Dense(1024, activation=None))
model.add(BN())
model.add(GN(0.3))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(1024, activation=None))
model.add(BN())
model.add(GN(0.3))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(1024, activation=None))
model.add(BN())
model.add(GN(0.3))
model.add(Activation('relu'))

model.add(Dense(num_classes, activation='softmax'))

model.summary()
model_dropout = model

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 784)               0         
_________________________________________________________________
gaussian_noise_4 (GaussianNo (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              803840    
_________________________________________________________________
batch_normalization_3 (Batch (None, 1024)              4096      
_________________________________________________________________
gaussian_noise_5 (GaussianNo (None, 1024)              0         
_________________________________________________________________
activation_3 (Activation)    (None, 1024)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1024)             

In [14]:
# Model BasicV2
model = Sequential()
model.add(Reshape(target_shape=(784,), input_shape=(28,28,1)))
model.add(GN(0.3))

model.add(Dense(512, activation=None))
model.add(BN())
model.add(GN(0.3))
model.add(Activation('relu'))

model.add(Dense(512, activation=None))
model.add(BN())
model.add(GN(0.3))
model.add(Activation('relu'))

model.add(Dense(512, activation=None))
model.add(BN())
model.add(GN(0.3))
model.add(Activation('relu'))

model.add(Dense(num_classes, activation='softmax'))

model.summary()
model_basicV2 = model

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 784)               0         
_________________________________________________________________
gaussian_noise (GaussianNois (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 512)               0         
_________________________________________________________________
activation (Activation)      (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               2

In [15]:
# Model BasicReduced
model = Sequential()
model.add(Reshape(target_shape=(784,), input_shape=(28,28,1)))
model.add(GN(0.3))

model.add(Dense(1024, activation=None))
model.add(BN())
model.add(GN(0.2))
model.add(Activation('relu'))

model.add(Dense(1024, activation=None))
model.add(BN())
model.add(GN(0.1))
model.add(Activation('relu'))

model.add(Dense(num_classes, activation='softmax'))

model.summary()
model_basicReduced = model

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 784)               0         
_________________________________________________________________
gaussian_noise_4 (GaussianNo (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              803840    
_________________________________________________________________
batch_normalization_3 (Batch (None, 1024)              4096      
_________________________________________________________________
gaussian_noise_5 (GaussianNo (None, 1024)              0         
_________________________________________________________________
activation_3 (Activation)    (None, 1024)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1024)             

In [16]:
# Model BasicReducedV2
model = Sequential()
model.add(Reshape(target_shape=(784,), input_shape=(28,28,1)))
model.add(GN(0.3))

model.add(Dense(512, activation=None))
model.add(BN())
model.add(GN(0.2))
model.add(Activation('relu'))

model.add(Dense(512, activation=None))
model.add(BN())
model.add(GN(0.1))
model.add(Activation('relu'))

model.add(Dense(num_classes, activation='softmax'))

model.summary()
model_basicReducedV2 = model

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 784)               0         
_________________________________________________________________
gaussian_noise_7 (GaussianNo (None, 784)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               401920    
_________________________________________________________________
batch_normalization_5 (Batch (None, 512)               2048      
_________________________________________________________________
gaussian_noise_8 (GaussianNo (None, 512)               0         
_________________________________________________________________
activation_5 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)              

In [18]:
# Model BasicExtended
model = Sequential()
model.add(Reshape(target_shape=(784,), input_shape=(28,28,1)))
model.add(GN(0.3))

model.add(Dense(4096, activation=None))
model.add(BN())
model.add(GN(0.3))
model.add(Activation('relu'))

model.add(Dense(2048, activation=None))
model.add(BN())
model.add(GN(0.3))
model.add(Activation('relu'))

model.add(Dense(1024, activation=None))
model.add(BN())
model.add(GN(0.3))
model.add(Activation('relu'))

model.add(Dense(num_classes, activation='softmax'))

model.summary()
model_basicExtended = model

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_3 (Reshape)          (None, 784)               0         
_________________________________________________________________
gaussian_noise_10 (GaussianN (None, 784)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 4096)              3215360   
_________________________________________________________________
batch_normalization_7 (Batch (None, 4096)              16384     
_________________________________________________________________
gaussian_noise_11 (GaussianN (None, 4096)              0         
_________________________________________________________________
activation_7 (Activation)    (None, 4096)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 2048)             

## Training

In [18]:
# Basic corrected and Basic training
model_5 = model_basic
model_5 = basic_training_and_evaluation(model=model_5,
                                        model_id='model_5',
                                        scheduler=scheduler_lr)

Epoch 1/200
937/937 [============================>.] - ETA: 0s - loss: 0.7616 - accuracy: 0.7982
Epoch 00001: val_accuracy improved from -inf to 0.95910, saving model to models/mlp-model_5-0001-0.959100.h5

Epoch 00001: saving model to models/mlp-model_5.h5
938/937 [==============================] - 19s 20ms/step - loss: 0.7612 - accuracy: 0.7983 - val_loss: 0.1286 - val_accuracy: 0.9591
Epoch 2/200
937/937 [============================>.] - ETA: 0s - loss: 0.3420 - accuracy: 0.8913
Epoch 00002: val_accuracy improved from 0.95910 to 0.97180, saving model to models/mlp-model_5-0002-0.971800.h5

Epoch 00002: saving model to models/mlp-model_5.h5
938/937 [==============================] - 22s 24ms/step - loss: 0.3419 - accuracy: 0.8913 - val_loss: 0.0874 - val_accuracy: 0.9718
Epoch 3/200
936/937 [============================>.] - ETA: 0s - loss: 0.2856 - accuracy: 0.9101
Epoch 00003: val_accuracy improved from 0.97180 to 0.97620, saving model to models/mlp-model_5-0003-0.976200.h5

Epoch

Epoch 45/200
936/937 [============================>.] - ETA: 0s - loss: 0.0838 - accuracy: 0.9725
Epoch 00045: val_accuracy did not improve from 0.99320

Epoch 00045: saving model to models/mlp-model_5.h5
938/937 [==============================] - 20s 21ms/step - loss: 0.0838 - accuracy: 0.9725 - val_loss: 0.0211 - val_accuracy: 0.9929
Epoch 46/200
938/937 [==============================] - ETA: 0s - loss: 0.0840 - accuracy: 0.9726
Epoch 00046: val_accuracy did not improve from 0.99320

Epoch 00046: saving model to models/mlp-model_5.h5
938/937 [==============================] - 20s 21ms/step - loss: 0.0840 - accuracy: 0.9726 - val_loss: 0.0206 - val_accuracy: 0.9930
Epoch 47/200
936/937 [============================>.] - ETA: 0s - loss: 0.0835 - accuracy: 0.9733
Epoch 00047: val_accuracy did not improve from 0.99320

Epoch 00047: saving model to models/mlp-model_5.h5
938/937 [==============================] - 21s 22ms/step - loss: 0.0835 - accuracy: 0.9732 - val_loss: 0.0206 - val_acc

Epoch 69/200
938/937 [==============================] - ETA: 0s - loss: 0.0808 - accuracy: 0.9739
Epoch 00069: val_accuracy did not improve from 0.99400

Epoch 00069: saving model to models/mlp-model_5.h5
938/937 [==============================] - 20s 21ms/step - loss: 0.0808 - accuracy: 0.9739 - val_loss: 0.0194 - val_accuracy: 0.9938
Epoch 70/200
937/937 [============================>.] - ETA: 0s - loss: 0.0794 - accuracy: 0.9739
Epoch 00070: val_accuracy did not improve from 0.99400

Epoch 00070: saving model to models/mlp-model_5.h5
938/937 [==============================] - 21s 22ms/step - loss: 0.0794 - accuracy: 0.9739 - val_loss: 0.0199 - val_accuracy: 0.9935
Epoch 71/200
938/937 [==============================] - ETA: 0s - loss: 0.0781 - accuracy: 0.9746
Epoch 00071: val_accuracy did not improve from 0.99400

Epoch 00071: saving model to models/mlp-model_5.h5
938/937 [==============================] - 20s 22ms/step - loss: 0.0781 - accuracy: 0.9746 - val_loss: 0.0194 - val_acc

936/937 [============================>.] - ETA: 0s - loss: 0.0762 - accuracy: 0.9753
Epoch 00117: val_accuracy did not improve from 0.99400

Epoch 00117: saving model to models/mlp-model_5.h5
938/937 [==============================] - 21s 22ms/step - loss: 0.0761 - accuracy: 0.9753 - val_loss: 0.0193 - val_accuracy: 0.9936
Epoch 118/200
937/937 [============================>.] - ETA: 0s - loss: 0.0780 - accuracy: 0.9746
Epoch 00118: val_accuracy did not improve from 0.99400

Epoch 00118: saving model to models/mlp-model_5.h5
938/937 [==============================] - 21s 22ms/step - loss: 0.0780 - accuracy: 0.9746 - val_loss: 0.0192 - val_accuracy: 0.9938
Epoch 119/200
936/937 [============================>.] - ETA: 0s - loss: 0.0750 - accuracy: 0.9753
Epoch 00119: val_accuracy did not improve from 0.99400

Epoch 00119: saving model to models/mlp-model_5.h5
938/937 [==============================] - 20s 21ms/step - loss: 0.0749 - accuracy: 0.9754 - val_loss: 0.0188 - val_accuracy: 0.99

937/937 [============================>.] - ETA: 0s - loss: 0.0744 - accuracy: 0.9762
Epoch 00165: val_accuracy did not improve from 0.99400

Epoch 00165: saving model to models/mlp-model_5.h5
938/937 [==============================] - 21s 22ms/step - loss: 0.0744 - accuracy: 0.9762 - val_loss: 0.0198 - val_accuracy: 0.9930
Epoch 166/200
937/937 [============================>.] - ETA: 0s - loss: 0.0751 - accuracy: 0.9753
Epoch 00166: val_accuracy did not improve from 0.99400

Epoch 00166: saving model to models/mlp-model_5.h5
938/937 [==============================] - 21s 22ms/step - loss: 0.0752 - accuracy: 0.9753 - val_loss: 0.0196 - val_accuracy: 0.9934
Epoch 167/200
938/937 [==============================] - ETA: 0s - loss: 0.0720 - accuracy: 0.9771
Epoch 00167: val_accuracy did not improve from 0.99400

Epoch 00167: saving model to models/mlp-model_5.h5
938/937 [==============================] - 20s 21ms/step - loss: 0.0720 - accuracy: 0.9771 - val_loss: 0.0192 - val_accuracy: 0.99

In [19]:
# Basic corrected and Advanced training
model_6 = model_basic
model_6 = advanced_training_and_evaluation(model=model_6,
                                        model_id='model_6',
                                        scheduler=reduce_lr)

Epoch 1/200
937/937 [============================>.] - ETA: 0s - loss: 0.1049 - accuracy: 0.9655
Epoch 00001: val_accuracy improved from -inf to 0.99400, saving model to models/mlp-model_6-0001-0.994000.h5

Epoch 00001: saving model to models/mlp-model_6.h5
938/937 [==============================] - 24s 25ms/step - loss: 0.1048 - accuracy: 0.9655 - val_loss: 0.0178 - val_accuracy: 0.9940
Epoch 2/200
937/937 [============================>.] - ETA: 0s - loss: 0.1029 - accuracy: 0.9667
Epoch 00002: val_accuracy improved from 0.99400 to 0.99420, saving model to models/mlp-model_6-0002-0.994200.h5

Epoch 00002: saving model to models/mlp-model_6.h5
938/937 [==============================] - 23s 25ms/step - loss: 0.1030 - accuracy: 0.9667 - val_loss: 0.0176 - val_accuracy: 0.9942
Epoch 3/200
937/937 [============================>.] - ETA: 0s - loss: 0.1037 - accuracy: 0.9662
Epoch 00003: val_accuracy did not improve from 0.99420

Epoch 00003: saving model to models/mlp-model_6.h5
938/937 [==

Epoch 24/200
937/937 [============================>.] - ETA: 0s - loss: 0.0996 - accuracy: 0.9680
Epoch 00024: val_accuracy did not improve from 0.99440

Epoch 00024: saving model to models/mlp-model_6.h5
938/937 [==============================] - 23s 25ms/step - loss: 0.0996 - accuracy: 0.9680 - val_loss: 0.0181 - val_accuracy: 0.9942
Epoch 25/200
937/937 [============================>.] - ETA: 0s - loss: 0.1016 - accuracy: 0.9674
Epoch 00025: val_accuracy did not improve from 0.99440

Epoch 00025: saving model to models/mlp-model_6.h5
938/937 [==============================] - 26s 27ms/step - loss: 0.1015 - accuracy: 0.9674 - val_loss: 0.0182 - val_accuracy: 0.9940
Epoch 26/200
937/937 [============================>.] - ETA: 0s - loss: 0.0990 - accuracy: 0.9685
Epoch 00026: val_accuracy did not improve from 0.99440

Epoch 00026: saving model to models/mlp-model_6.h5
938/937 [==============================] - 26s 27ms/step - loss: 0.0990 - accuracy: 0.9685 - val_loss: 0.0182 - val_acc

938/937 [==============================] - ETA: 0s - loss: 0.0979 - accuracy: 0.9686
Epoch 00072: val_accuracy did not improve from 0.99440

Epoch 00072: saving model to models/mlp-model_6.h5
938/937 [==============================] - 23s 24ms/step - loss: 0.0979 - accuracy: 0.9686 - val_loss: 0.0182 - val_accuracy: 0.9942
Epoch 73/200
938/937 [==============================] - ETA: 0s - loss: 0.0976 - accuracy: 0.9685
Epoch 00073: val_accuracy did not improve from 0.99440

Epoch 00073: saving model to models/mlp-model_6.h5
938/937 [==============================] - 25s 26ms/step - loss: 0.0976 - accuracy: 0.9685 - val_loss: 0.0184 - val_accuracy: 0.9941
Epoch 74/200
938/937 [==============================] - ETA: 0s - loss: 0.0937 - accuracy: 0.9687
Epoch 00074: val_accuracy did not improve from 0.99440

Epoch 00074: saving model to models/mlp-model_6.h5
938/937 [==============================] - 24s 25ms/step - loss: 0.0937 - accuracy: 0.9687 - val_loss: 0.0181 - val_accuracy: 0.9939

936/937 [============================>.] - ETA: 0s - loss: 0.0926 - accuracy: 0.9697
Epoch 00120: val_accuracy did not improve from 0.99450

Epoch 00120: saving model to models/mlp-model_6.h5
938/937 [==============================] - 25s 27ms/step - loss: 0.0925 - accuracy: 0.9697 - val_loss: 0.0184 - val_accuracy: 0.9941
Epoch 121/200
937/937 [============================>.] - ETA: 0s - loss: 0.0909 - accuracy: 0.9708
Epoch 00121: val_accuracy did not improve from 0.99450

Epoch 00121: saving model to models/mlp-model_6.h5
938/937 [==============================] - 25s 27ms/step - loss: 0.0909 - accuracy: 0.9707 - val_loss: 0.0185 - val_accuracy: 0.9939
Epoch 122/200
938/937 [==============================] - ETA: 0s - loss: 0.0934 - accuracy: 0.9694
Epoch 00122: val_accuracy did not improve from 0.99450

Epoch 00122: saving model to models/mlp-model_6.h5
938/937 [==============================] - 25s 27ms/step - loss: 0.0934 - accuracy: 0.9694 - val_loss: 0.0187 - val_accuracy: 0.99

938/937 [==============================] - ETA: 0s - loss: 0.0927 - accuracy: 0.9702
Epoch 00168: val_accuracy did not improve from 0.99450

Epoch 00168: saving model to models/mlp-model_6.h5
938/937 [==============================] - 25s 26ms/step - loss: 0.0927 - accuracy: 0.9702 - val_loss: 0.0186 - val_accuracy: 0.9941
Epoch 169/200
938/937 [==============================] - ETA: 0s - loss: 0.0903 - accuracy: 0.9697 ETA: 0s - l
Epoch 00169: val_accuracy did not improve from 0.99450

Epoch 00169: saving model to models/mlp-model_6.h5
938/937 [==============================] - 23s 25ms/step - loss: 0.0903 - accuracy: 0.9697 - val_loss: 0.0184 - val_accuracy: 0.9941
Epoch 170/200
937/937 [============================>.] - ETA: 0s - loss: 0.0918 - accuracy: 0.9695
Epoch 00170: val_accuracy did not improve from 0.99450

Epoch 00170: saving model to models/mlp-model_6.h5
938/937 [==============================] - 24s 25ms/step - loss: 0.0918 - accuracy: 0.9695 - val_loss: 0.0183 - val_ac

In [21]:
# Basic corrected, scheduler 2 and Basic training
model_7 = model_basic
model_7 = basic_training_and_evaluation(model=model_7,
                                        model_id='model_7',
                                        scheduler=scheduler_lr_2)

Epoch 1/200
938/937 [==============================] - ETA: 0s - loss: 0.0874 - accuracy: 0.9709
Epoch 00001: val_accuracy improved from -inf to 0.99230, saving model to models/mlp-model_7-0001-0.992300.h5

Epoch 00001: saving model to models/mlp-model_7.h5
938/937 [==============================] - 21s 22ms/step - loss: 0.0874 - accuracy: 0.9709 - val_loss: 0.0231 - val_accuracy: 0.9923
Epoch 2/200
938/937 [==============================] - ETA: 0s - loss: 0.0883 - accuracy: 0.9718
Epoch 00002: val_accuracy did not improve from 0.99230

Epoch 00002: saving model to models/mlp-model_7.h5
938/937 [==============================] - 21s 23ms/step - loss: 0.0883 - accuracy: 0.9718 - val_loss: 0.0248 - val_accuracy: 0.9921
Epoch 3/200
936/937 [============================>.] - ETA: 0s - loss: 0.0893 - accuracy: 0.9714
Epoch 00003: val_accuracy did not improve from 0.99230

Epoch 00003: saving model to models/mlp-model_7.h5
938/937 [==============================] - 21s 23ms/step - loss: 0.0

938/937 [==============================] - 21s 23ms/step - loss: 0.0760 - accuracy: 0.9753 - val_loss: 0.0210 - val_accuracy: 0.9942
Epoch 24/200
937/937 [============================>.] - ETA: 0s - loss: 0.0757 - accuracy: 0.9751
Epoch 00024: val_accuracy did not improve from 0.99420

Epoch 00024: saving model to models/mlp-model_7.h5
938/937 [==============================] - 21s 22ms/step - loss: 0.0758 - accuracy: 0.9751 - val_loss: 0.0212 - val_accuracy: 0.9930
Epoch 25/200
938/937 [==============================] - ETA: 0s - loss: 0.0776 - accuracy: 0.9755
Epoch 00025: val_accuracy did not improve from 0.99420

Epoch 00025: saving model to models/mlp-model_7.h5
938/937 [==============================] - 21s 22ms/step - loss: 0.0776 - accuracy: 0.9755 - val_loss: 0.0230 - val_accuracy: 0.9925
Epoch 26/200
937/937 [============================>.] - ETA: 0s - loss: 0.0717 - accuracy: 0.9768
Epoch 00026: val_accuracy did not improve from 0.99420

Epoch 00026: saving model to models/m

Epoch 48/200
938/937 [==============================] - ETA: 0s - loss: 0.0582 - accuracy: 0.9815
Epoch 00048: val_accuracy did not improve from 0.99430

Epoch 00048: saving model to models/mlp-model_7.h5
938/937 [==============================] - 21s 22ms/step - loss: 0.0582 - accuracy: 0.9815 - val_loss: 0.0179 - val_accuracy: 0.9941
Epoch 49/200
937/937 [============================>.] - ETA: 0s - loss: 0.0598 - accuracy: 0.9806
Epoch 00049: val_accuracy did not improve from 0.99430

Epoch 00049: saving model to models/mlp-model_7.h5
938/937 [==============================] - 21s 22ms/step - loss: 0.0597 - accuracy: 0.9806 - val_loss: 0.0178 - val_accuracy: 0.9941
Epoch 50/200
938/937 [==============================] - ETA: 0s - loss: 0.0561 - accuracy: 0.9818
Epoch 00050: val_accuracy did not improve from 0.99430

Epoch 00050: saving model to models/mlp-model_7.h5
938/937 [==============================] - 21s 22ms/step - loss: 0.0561 - accuracy: 0.9818 - val_loss: 0.0182 - val_acc

936/937 [============================>.] - ETA: 0s - loss: 0.0553 - accuracy: 0.9820
Epoch 00096: val_accuracy did not improve from 0.99460

Epoch 00096: saving model to models/mlp-model_7.h5
938/937 [==============================] - 21s 23ms/step - loss: 0.0552 - accuracy: 0.9820 - val_loss: 0.0175 - val_accuracy: 0.9944
Epoch 97/200
936/937 [============================>.] - ETA: 0s - loss: 0.0542 - accuracy: 0.9818
Epoch 00097: val_accuracy did not improve from 0.99460

Epoch 00097: saving model to models/mlp-model_7.h5
938/937 [==============================] - 21s 23ms/step - loss: 0.0544 - accuracy: 0.9817 - val_loss: 0.0176 - val_accuracy: 0.9941
Epoch 98/200
938/937 [==============================] - ETA: 0s - loss: 0.0569 - accuracy: 0.9813
Epoch 00098: val_accuracy did not improve from 0.99460

Epoch 00098: saving model to models/mlp-model_7.h5
938/937 [==============================] - 21s 22ms/step - loss: 0.0569 - accuracy: 0.9813 - val_loss: 0.0175 - val_accuracy: 0.9944

937/937 [============================>.] - ETA: 0s - loss: 0.0544 - accuracy: 0.9821
Epoch 00144: val_accuracy did not improve from 0.99460

Epoch 00144: saving model to models/mlp-model_7.h5
938/937 [==============================] - 21s 22ms/step - loss: 0.0545 - accuracy: 0.9821 - val_loss: 0.0173 - val_accuracy: 0.9940
Epoch 145/200
937/937 [============================>.] - ETA: 0s - loss: 0.0535 - accuracy: 0.9828
Epoch 00145: val_accuracy did not improve from 0.99460

Epoch 00145: saving model to models/mlp-model_7.h5
938/937 [==============================] - 20s 21ms/step - loss: 0.0535 - accuracy: 0.9828 - val_loss: 0.0174 - val_accuracy: 0.9940
Epoch 146/200
938/937 [==============================] - ETA: 0s - loss: 0.0562 - accuracy: 0.9819
Epoch 00146: val_accuracy did not improve from 0.99460

Epoch 00146: saving model to models/mlp-model_7.h5
938/937 [==============================] - 20s 21ms/step - loss: 0.0562 - accuracy: 0.9819 - val_loss: 0.0175 - val_accuracy: 0.99

938/937 [==============================] - ETA: 0s - loss: 0.0542 - accuracy: 0.9824
Epoch 00192: val_accuracy did not improve from 0.99460

Epoch 00192: saving model to models/mlp-model_7.h5
938/937 [==============================] - 20s 21ms/step - loss: 0.0542 - accuracy: 0.9824 - val_loss: 0.0173 - val_accuracy: 0.9942
Epoch 193/200
938/937 [==============================] - ETA: 0s - loss: 0.0557 - accuracy: 0.9820
Epoch 00193: val_accuracy did not improve from 0.99460

Epoch 00193: saving model to models/mlp-model_7.h5
938/937 [==============================] - 20s 21ms/step - loss: 0.0557 - accuracy: 0.9820 - val_loss: 0.0173 - val_accuracy: 0.9944
Epoch 194/200
937/937 [============================>.] - ETA: 0s - loss: 0.0533 - accuracy: 0.9828
Epoch 00194: val_accuracy did not improve from 0.99460

Epoch 00194: saving model to models/mlp-model_7.h5
938/937 [==============================] - 21s 22ms/step - loss: 0.0534 - accuracy: 0.9828 - val_loss: 0.0170 - val_accuracy: 0.99

In [22]:
# Basic corrected, scheduler 2 and Advanced training
model_8 = model_basic
model_8 = advanced_training_and_evaluation(model=model_8,
                                        model_id='model_8',
                                        scheduler=scheduler_lr_2)

Epoch 1/200
938/937 [==============================] - ETA: 0s - loss: 0.0890 - accuracy: 0.9714
Epoch 00001: val_accuracy improved from -inf to 0.99440, saving model to models/mlp-model_8-0001-0.994400.h5

Epoch 00001: saving model to models/mlp-model_8.h5
938/937 [==============================] - 24s 26ms/step - loss: 0.0890 - accuracy: 0.9714 - val_loss: 0.0190 - val_accuracy: 0.9944
Epoch 2/200
937/937 [============================>.] - ETA: 0s - loss: 0.0933 - accuracy: 0.9700
Epoch 00002: val_accuracy did not improve from 0.99440

Epoch 00002: saving model to models/mlp-model_8.h5
938/937 [==============================] - 23s 25ms/step - loss: 0.0932 - accuracy: 0.9700 - val_loss: 0.0245 - val_accuracy: 0.9924
Epoch 3/200
936/937 [============================>.] - ETA: 0s - loss: 0.0898 - accuracy: 0.9712
Epoch 00003: val_accuracy did not improve from 0.99440

Epoch 00003: saving model to models/mlp-model_8.h5
938/937 [==============================] - 25s 26ms/step - loss: 0.0

Epoch 25/200
937/937 [============================>.] - ETA: 0s - loss: 0.0813 - accuracy: 0.9735
Epoch 00025: val_accuracy did not improve from 0.99440

Epoch 00025: saving model to models/mlp-model_8.h5
938/937 [==============================] - 24s 25ms/step - loss: 0.0812 - accuracy: 0.9736 - val_loss: 0.0220 - val_accuracy: 0.9933
Epoch 26/200
938/937 [==============================] - ETA: 0s - loss: 0.0754 - accuracy: 0.9754
Epoch 00026: val_accuracy did not improve from 0.99440

Epoch 00026: saving model to models/mlp-model_8.h5
938/937 [==============================] - 23s 25ms/step - loss: 0.0754 - accuracy: 0.9754 - val_loss: 0.0210 - val_accuracy: 0.9938
Epoch 27/200
938/937 [==============================] - ETA: 0s - loss: 0.0758 - accuracy: 0.9747
Epoch 00027: val_accuracy did not improve from 0.99440

Epoch 00027: saving model to models/mlp-model_8.h5
938/937 [==============================] - 24s 26ms/step - loss: 0.0758 - accuracy: 0.9747 - val_loss: 0.0204 - val_acc

Epoch 49/200
937/937 [============================>.] - ETA: 0s - loss: 0.0638 - accuracy: 0.9797
Epoch 00049: val_accuracy did not improve from 0.99480

Epoch 00049: saving model to models/mlp-model_8.h5
938/937 [==============================] - 23s 24ms/step - loss: 0.0638 - accuracy: 0.9798 - val_loss: 0.0180 - val_accuracy: 0.9945
Epoch 50/200
937/937 [============================>.] - ETA: 0s - loss: 0.0636 - accuracy: 0.9794
Epoch 00050: val_accuracy did not improve from 0.99480

Epoch 00050: saving model to models/mlp-model_8.h5
938/937 [==============================] - 23s 25ms/step - loss: 0.0636 - accuracy: 0.9794 - val_loss: 0.0176 - val_accuracy: 0.9940
Epoch 51/200
937/937 [============================>.] - ETA: 0s - loss: 0.0645 - accuracy: 0.9783
Epoch 00051: val_accuracy did not improve from 0.99480

Epoch 00051: saving model to models/mlp-model_8.h5
938/937 [==============================] - 25s 26ms/step - loss: 0.0645 - accuracy: 0.9783 - val_loss: 0.0177 - val_acc

936/937 [============================>.] - ETA: 0s - loss: 0.0604 - accuracy: 0.9790
Epoch 00097: val_accuracy did not improve from 0.99480

Epoch 00097: saving model to models/mlp-model_8.h5
938/937 [==============================] - 23s 25ms/step - loss: 0.0605 - accuracy: 0.9789 - val_loss: 0.0174 - val_accuracy: 0.9943
Epoch 98/200
938/937 [==============================] - ETA: 0s - loss: 0.0636 - accuracy: 0.9791
Epoch 00098: val_accuracy did not improve from 0.99480

Epoch 00098: saving model to models/mlp-model_8.h5
938/937 [==============================] - 22s 24ms/step - loss: 0.0636 - accuracy: 0.9791 - val_loss: 0.0174 - val_accuracy: 0.9943
Epoch 99/200
937/937 [============================>.] - ETA: 0s - loss: 0.0631 - accuracy: 0.9793
Epoch 00099: val_accuracy did not improve from 0.99480

Epoch 00099: saving model to models/mlp-model_8.h5
938/937 [==============================] - 25s 26ms/step - loss: 0.0632 - accuracy: 0.9793 - val_loss: 0.0174 - val_accuracy: 0.9945

938/937 [==============================] - ETA: 0s - loss: 0.0607 - accuracy: 0.9799
Epoch 00145: val_accuracy did not improve from 0.99480

Epoch 00145: saving model to models/mlp-model_8.h5
938/937 [==============================] - 23s 25ms/step - loss: 0.0607 - accuracy: 0.9799 - val_loss: 0.0172 - val_accuracy: 0.9942
Epoch 146/200
937/937 [============================>.] - ETA: 0s - loss: 0.0626 - accuracy: 0.9797
Epoch 00146: val_accuracy did not improve from 0.99480

Epoch 00146: saving model to models/mlp-model_8.h5
938/937 [==============================] - 23s 25ms/step - loss: 0.0626 - accuracy: 0.9797 - val_loss: 0.0173 - val_accuracy: 0.9942
Epoch 147/200
938/937 [==============================] - ETA: 0s - loss: 0.0596 - accuracy: 0.9799
Epoch 00147: val_accuracy did not improve from 0.99480

Epoch 00147: saving model to models/mlp-model_8.h5
938/937 [==============================] - 23s 25ms/step - loss: 0.0596 - accuracy: 0.9799 - val_loss: 0.0175 - val_accuracy: 0.99

936/937 [============================>.] - ETA: 0s - loss: 0.0633 - accuracy: 0.9795
Epoch 00193: val_accuracy did not improve from 0.99480

Epoch 00193: saving model to models/mlp-model_8.h5
938/937 [==============================] - 23s 25ms/step - loss: 0.0632 - accuracy: 0.9795 - val_loss: 0.0174 - val_accuracy: 0.9942
Epoch 194/200
937/937 [============================>.] - ETA: 0s - loss: 0.0621 - accuracy: 0.9801
Epoch 00194: val_accuracy did not improve from 0.99480

Epoch 00194: saving model to models/mlp-model_8.h5
938/937 [==============================] - 24s 25ms/step - loss: 0.0621 - accuracy: 0.9801 - val_loss: 0.0175 - val_accuracy: 0.9941
Epoch 195/200
934/937 [============================>.] - ETA: 0s - loss: 0.0614 - accuracy: 0.9802
Epoch 00195: val_accuracy did not improve from 0.99480

Epoch 00195: saving model to models/mlp-model_8.h5
938/937 [==============================] - 24s 25ms/step - loss: 0.0613 - accuracy: 0.9802 - val_loss: 0.0174 - val_accuracy: 0.99